# 1. identity extraction

In [7]:
import os
import argparse
import numpy as np 
from tqdm import  tqdm
from glob import glob
from utils.libs import load_image, get_frame_id
from config.config import Config
from torch.nn import DataParallel
from models import resnet_face18
import torch
import cv2
torch.cuda.empty_cache()

test_model_path= "./pretrained/resnet18_110.pth"
opt = Config()
model = resnet_face18(opt.use_se)
model = DataParallel(model)
model.load_state_dict(torch.load(test_model_path))
model.to(torch.device("cuda"))
model.eval()

def video_id_extraction(path, 
                        save, 
                        len_save, 
                        batch_size = 16,
                        compress=None, 
                        noise=None):
    whole_set_ids = None
    video_length = []
    video_list = glob(path+"/*")
    video_list.sort()
    for video in tqdm(video_list):
        batch_frames = None
        video_ids = None
        batch_count = 0
        video_frames = glob(video+"/*")
        if len(glob(video+"/*")) < 1:
          print("empty folder", video)
          continue
        frame_list = glob(video+"/*")
        for frame in frame_list:
            batch_count += 1
            try:
             single_image = load_image(frame, compress=compress, noise=noise)
            except Exception as e:
              print("blocked-", frame)
              print("error-",e)
              del e
              continue
            if batch_frames is None:
                batch_frames = single_image
            else:
                batch_frames = np.concatenate((batch_frames, single_image), axis=0)
            if not batch_count % batch_size or batch_count==len(video_frames):
                batch_ids = get_frame_id(model, batch_frames, batch_size=batch_size)
                batch_frames = None
                if video_ids is None:
                    video_ids = batch_ids
                else:
                    video_len1 = video_ids.shape[0]
                    video_ids = np.concatenate((video_ids, batch_ids), axis=0)
        if whole_set_ids is None:
            whole_set_ids = video_ids
        else:
            whole_set_ids = np.concatenate((whole_set_ids, video_ids), axis=0)
        video_length.append(len(video_frames))
    video_length = np.array(video_length) 
    print("video_length", sum(video_length))
    print("whole_set_ids", len(whole_set_ids))
    np.save(save, whole_set_ids)
    np.save(len_save,video_length)
    del whole_set_ids, video_length

### 1.1. FaceForensics++  dataset

In [ ]:
save_root = "./id_vectors/FFpp/"

paths=["/home/baoping/FaceForensics++/manipulated_sequences/NeuralTextures/raw/aligned_faces", \
       "/home/baoping/FaceForensics++/manipulated_sequences/FaceSwap/raw/aligned_faces", \
       "/home/baoping/FaceForensics++/manipulated_sequences/Face2Face/raw/aligned_faces", \
       "/home/baoping/FaceForensics++/manipulated_sequences/Deepfakes/raw/aligned_faces", \
       "/home/baoping/FaceForensics++/original_sequences/youtube/raw/aligned_faces"  ]

ids_save_path = ["ffpp_nt_ids.npy", "ffpp_fs_ids.npy", "ffpp_ff_ids.npy", "ffpp_df_ids.npy", "ffpp_real_ids.npy"]#
len_save_path = ["ffpp_nt_lens.npy", "ffpp_fs_lens.npy", "ffpp_ff_lens.npy", "ffpp_df_lens.npy", "ffpp_real_lens.npy"]#
for idx, p in enumerate(paths):
    video_id_extraction(p, 
                        os.path.join( ids_save_path[idx] ), 
                        os.path.join( len_save_path[idx]) )

In [ ]:
#merge several fake datasets
import numpy as np
import os

def set_merge(fake_id_list, real_length):
    fake_type_count = len(fake_id_list)
    real_video_count = real_length
    fake_video_count = int(real_video_count // fake_type_count)
    print("video count", real_video_count, fake_video_count)
    ret_list = []
    for i in fake_id_list:
        ret_list += i[:fake_video_count]
    return ret_list
ffpp_real_ids = "./id_vectors/FFpp/ffpp_real_ids.npy"
ffpp_real_len = "./id_vectors/FFpp/ffpp_real_lens.npy"
ffpp_df_ids = "./id_vectors/FFpp/ffpp_df_ids.npy"
ffpp_df_len = "./id_vectors/FFpp/ffpp_df_lens.npy"
ffpp_ff_ids = "./id_vectors/FFpp/ffpp_ff_ids.npy"
ffpp_ff_len = "./id_vectors/FFpp/ffpp_ff_lens.npy"
ffpp_fs_ids = "./id_vectors/FFpp/ffpp_fs_ids.npy"
ffpp_fs_len = "./id_vectors/FFpp/ffpp_fs_lens.npy"
ffpp_nt_ids = "./id_vectors/FFpp/ffpp_nt_ids.npy"
ffpp_nt_len = "./id_vectors/FFpp/ffpp_nt_lens.npy"

fake_ids_list = [ffpp_df_ids, ffpp_ff_ids, ffpp_fs_ids, ffpp_nt_ids]
fake_len_list = [ffpp_df_len, ffpp_ff_len, ffpp_fs_len, ffpp_nt_len]

real_ids = np.load(ffpp_real_ids)
real_len = np.load(ffpp_real_len, allow_pickle=True)
print("real shapes", real_ids.shape, real_len.shape)

merged_ids = None
merged_len = None
for i,l in zip(fake_ids_list,fake_len_list):#range(4):
    fake_ids = np.load(i)
    fake_len = np.load(l, allow_pickle=True)
    if merged_ids is None:
        merged_ids = fake_ids
        merged_len = fake_len
    else:
        merged_ids = np.concatenate((merged_ids,fake_ids))
        merged_len = np.concatenate((merged_len,fake_len))

np.save("./id_vectors/FFpp/ffpp_fake_ids.npy", merged_ids)
np.save("./id_vectors/FFpp/ffpp_fake_len.npy", merged_len)

### 1.2. CelebDF

In [ ]:
save_root = "./id_vectors/CelebDF/"

paths=["/media/baoping/8TSGHDD/CelebDF/Celeb-real/aligned_faces", \
       "/media/baoping/8TSGHDD/CelebDF/Celeb-synthesis/aligned_faces"]

ids_save_path = ["celebdf_real.npy", "celebdf_fake.npy"]
len_save_path = ["celebdf_real_lens.npy", "celebdf_fake_lens.npy"]
for idx, p in enumerate(paths):
    video_id_extraction(p, 
                        os.path.join(save_root, ids_save_path[idx]), 
                        os.path.join(save_root, len_save_path[idx]))

### 1.3. DFD

In [ ]:
save_root = "./id_vectors/DFD/"

paths=["/media/baoping/8TSGHDD/DeepFakeDetection/real", \
       "/media/baoping/8TSGHDD/DeepFakeDetection/fake"]

ids_save_path = ["dfd_real.npy", "dfd_fake.npy"]
len_save_path = ["dfd_real_lens.npy", "dfd_fake_lens.npy"]
for idx, p in enumerate(paths):
    video_id_extraction(p, 
                        os.path.join(save_root, ids_save_path[idx]), 
                        os.path.join(save_root, len_save_path[idx]))

### 1.4.Deeper

In [ ]:
save_root = "./id_vectors/Deeper/"

paths=["/media/baoping/8TSGHDD/DeeperForensics1.0/manipulated_videos/end_to_end"]

ids_save_path = ["deeper_fake.npy"]
len_save_path = ["deeper_fake_lens.npy"]
for idx, p in enumerate(paths):
    video_id_extraction(p, 
                        os.path.join(save_root, ids_save_path[idx]), 
                        os.path.join(save_root, len_save_path[idx]))

# 2. TI2Net training

In [1]:
!python train.py \
--fake_selection="all" \
--model="test" \
--train_contrastive=1 \
--real_path="./id_vectors/real_ids.npy" \
--real_len_path="./id_vectors/real_lens.npy" \
--fake_path="./id_vectors/fake_ids.npy" \
--fake_len_path="./id_vectors/fake_lens.npy"

Using device: cuda:0
train set and test and set get
Training contrastive module
model TI2Net(
  (dropout_landmark): LandmarkDropout()
  (rnn): GRU(1024, 512, batch_first=True, bidirectional=True)
  (dropout_feature): Dropout(p=0.7, inplace=False)
  (linear_1): Linear(in_features=1024, out_features=128, bias=True)
  (relu): ReLU()
  (linear_2): Linear(in_features=128, out_features=2, bias=True)
  (output): Softmax(dim=1)
)
Using TripletMarginLoss
train iteration each epoch: 3000
test iteration each epoch: 1000
***************************************************************************************************
epoch:1 training: emb_loss:0.5005 clf_loss:0.65, loss:0.655, train_acc:0.6264save the model to ./weights/torch/test.pth
epoch:1 testing: test_seq_acc:0.562, test_seq_auc:0.4417, test_video_acc:0.562, test_video_auc:0.562save the model to ./weights/torch/test.pth
epoch:1 summary: best_seq_acc:0.562 best_seq_auc:0.4417 best_video_acc:0.562 best_video_auc:0.562save the model to ./weigh

***************************************************************************************************
epoch:14 training: emb_loss:0.3459 clf_loss:0.5388, loss:0.5423, train_acc:0.7511
epoch:14 testing: test_seq_acc:0.839, test_seq_auc:0.9764, test_video_acc:0.839, test_video_auc:0.9764
epoch:14 summary: best_seq_acc:0.905 best_seq_auc:0.9845 best_video_acc:0.905 best_video_auc:0.9845
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:15 training: emb_loss:0.3245 clf_loss:0.5313, loss:0.5345, train_acc:0.7593save the model to ./weights/torch/test.pth
epoch:15 testing: test_seq_acc:0.9055, test_seq_auc:0.9863, test_video_acc:0.9055, test_video_auc:0.9863save the model to ./weights/torch/test.pth
epoch:15 summary: best_seq_acc:0.9055 best_seq_auc:0.9863 best

***************************************************************************************************
epoch:28 training: emb_loss:0.2138 clf_loss:0.4759, loss:0.478, train_acc:0.8288
epoch:28 testing: test_seq_acc:0.8995, test_seq_auc:0.981, test_video_acc:0.8995, test_video_auc:0.981
epoch:28 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:29 training: emb_loss:0.2266 clf_loss:0.4805, loss:0.4828, train_acc:0.8203
epoch:29 testing: test_seq_acc:0.882, test_seq_auc:0.9888, test_video_acc:0.882, test_video_auc:0.9888
epoch:29 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
*************************************************

***************************************************************************************************
epoch:43 training: emb_loss:0.1892 clf_loss:0.4566, loss:0.4585, train_acc:0.8492
epoch:43 testing: test_seq_acc:0.9025, test_seq_auc:0.975, test_video_acc:0.9025, test_video_auc:0.975
epoch:43 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:44 training: emb_loss:0.1488 clf_loss:0.4501, loss:0.4516, train_acc:0.8556
epoch:44 testing: test_seq_acc:0.8085, test_seq_auc:0.9714, test_video_acc:0.8085, test_video_auc:0.9714
epoch:44 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
**********************************************

***************************************************************************************************
epoch:58 training: emb_loss:0.1147 clf_loss:0.435, loss:0.4361, train_acc:0.872 
epoch:58 testing: test_seq_acc:0.653, test_seq_auc:0.9032, test_video_acc:0.653, test_video_auc:0.9032
epoch:58 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:59 training: emb_loss:0.1266 clf_loss:0.4345, loss:0.4357, train_acc:0.8736
epoch:59 testing: test_seq_acc:0.785, test_seq_auc:0.908, test_video_acc:0.785, test_video_auc:0.908
epoch:59 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
***************************************************

...

***************************************************************************************************
epoch:133 training: emb_loss:0.05709 clf_loss:0.3868, loss:0.3874, train_acc:0.9241
epoch:133 testing: test_seq_acc:0.555, test_seq_auc:0.7473, test_video_acc:0.555, test_video_auc:0.7473
epoch:133 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:134 training: emb_loss:0.05616 clf_loss:0.3881, loss:0.3887, train_acc:0.9238
epoch:134 testing: test_seq_acc:0.5655, test_seq_auc:0.7767, test_video_acc:0.5655, test_video_auc:0.7767
epoch:134 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
**************************************

***************************************************************************************************
epoch:148 training: emb_loss:0.05132 clf_loss:0.3869, loss:0.3874, train_acc:0.9237
epoch:148 testing: test_seq_acc:0.503, test_seq_auc:0.7521, test_video_acc:0.503, test_video_auc:0.7521
epoch:148 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:149 training: emb_loss:0.05893 clf_loss:0.386, loss:0.3866, train_acc:0.9239
epoch:149 testing: test_seq_acc:0.513, test_seq_auc:0.7419, test_video_acc:0.513, test_video_auc:0.7419
epoch:149 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
*****************************************

***************************************************************************************************
epoch:163 training: emb_loss:0.05476 clf_loss:0.3815, loss:0.382, train_acc:0.9274
epoch:163 testing: test_seq_acc:0.5445, test_seq_auc:0.6754, test_video_acc:0.5445, test_video_auc:0.6754
epoch:163 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:164 training: emb_loss:0.04714 clf_loss:0.383, loss:0.3834, train_acc:0.9273
epoch:164 testing: test_seq_acc:0.522, test_seq_auc:0.6825, test_video_acc:0.522, test_video_auc:0.6825
epoch:164 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
****************************************

***************************************************************************************************
epoch:178 training: emb_loss:0.04228 clf_loss:0.3773, loss:0.3777, train_acc:0.9336
epoch:178 testing: test_seq_acc:0.514, test_seq_auc:0.6276, test_video_acc:0.514, test_video_auc:0.6276
epoch:178 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:179 training: emb_loss:0.04931 clf_loss:0.3742, loss:0.3747, train_acc:0.9359
epoch:179 testing: test_seq_acc:0.506, test_seq_auc:0.5458, test_video_acc:0.506, test_video_auc:0.5458
epoch:179 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
****************************************

***************************************************************************************************
epoch:193 training: emb_loss:0.05365 clf_loss:0.3779, loss:0.3785, train_acc:0.9331
epoch:193 testing: test_seq_acc:0.5145, test_seq_auc:0.6053, test_video_acc:0.5145, test_video_auc:0.6053
epoch:193 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:194 training: emb_loss:0.04582 clf_loss:0.372, loss:0.3725, train_acc:0.9377
epoch:194 testing: test_seq_acc:0.512, test_seq_auc:0.5048, test_video_acc:0.512, test_video_auc:0.512
epoch:194 summary: best_seq_acc:0.941 best_seq_auc:0.9945 best_video_acc:0.941 best_video_auc:0.9945
****************************************

# 3. TI2Net evaluation

In [11]:
!python evaluate.py \
--model="test.pth" \
--real_id_path="./id_vectors/FFpp/ffpp_real_ids_test.npy" \
--real_len_path="./id_vectors/FFpp/ffpp_real_lens_test.npy" \
--fake_id_path="./id_vectors/FFpp/ffpp_fake_ids_test.npy" \
--fake_len_path="./id_vectors/FFpp/ffpp_fake_lens_test.npy"

Using device: cuda:0
Please register the status.


#----Evaluation Results----#
Accuracy (sequence-level): 0.99
AUC (sequence-level): 0.998
Accuracy (video-level): 0.99
AUC (video-level): 0.998
#------------End-----------#
